# CSE6250BDH Deep Learning Labs
## 1. Feed-forward Neural Network

In this chapter, we will learn how to implement a feed-forward neural network by using PyTorch.
Before moving to neural networks, let's refresh the modeling with Scikit-learn that we have already done in the lab [Spark-mllib](http://www.sunlab.org/teaching/cse6250/fall2017/lab/spark-mllib/#Scikit-learn) and we will compare the results. If you have not completed that part, please complete it first.

### SVM

In [1]:
from sklearn.model_selection import train_test_split
from sklearn.datasets import load_svmlight_file
from sklearn.preprocessing import MaxAbsScaler

X, y = load_svmlight_file("patients.svmlight")
X = X.toarray() # make it dense

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=41)

In [2]:
scaler = MaxAbsScaler().fit(X_train)
X_train_transformed = scaler.transform(X_train)
X_test_transformed = scaler.transform(X_test)

In [3]:
from sklearn.svm import SVC
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import uniform as sp_rand

In [4]:
# CV
param_dist = {'C': sp_rand(0, 1000), 'kernel': ['linear', 'poly', 'rbf'], 'class_weight': [None, 'balanced']}

svm_cv = RandomizedSearchCV(estimator=SVC(), param_distributions=param_dist, scoring='roc_auc', cv=10, n_iter=10, n_jobs=-1, verbose=1, random_state=42)
svm_cv.fit(X_train_transformed, y_train)

Fitting 10 folds for each of 10 candidates, totalling 100 fits


[Parallel(n_jobs=-1)]: Done  74 out of 100 | elapsed:    6.2s remaining:    2.2s
[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed:    6.6s finished


RandomizedSearchCV(cv=10, error_score='raise',
          estimator=SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape=None, degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False),
          fit_params={}, iid=True, n_iter=10, n_jobs=-1,
          param_distributions={'C': <scipy.stats._distn_infrastructure.rv_frozen object at 0x7fd84b14ce10>, 'kernel': ['linear', 'poly', 'rbf'], 'class_weight': [None, 'balanced']},
          pre_dispatch='2*n_jobs', random_state=42, refit=True,
          return_train_score=True, scoring='roc_auc', verbose=1)

In [5]:
# Full training
svm_full = SVC().set_params(**(svm_cv.best_estimator_.get_params()))
svm_full.set_params(probability=True)
svm_full.fit(X_train_transformed, y_train)

y_pred, y_score = svm_full.predict(X_test_transformed), svm_full.predict_proba(X_test_transformed)
y_score = y_score[:, 1]

In [6]:
from sklearn.metrics import roc_curve, auc
fpr, tpr, _ = roc_curve(y_test, y_score)
auc_svm = auc(fpr, tpr)

In [7]:
auc_svm

0.81812169312169325

### Feedforward Neural Network

Now, we will train a feed-forward neural network. We will do the following steps in order:

1. Load the training and test datasets using DataLoader
2. Define a Feedforwad Neural Network
3. Define a loss function
4. Train the network on the training data
5. Test the network on the test data

#### 1. Loading datasets
We will use DataLoader and TensorDataset (from [torch.utils.data](http://pytorch.org/docs/master/data.html#)) for convinience in data handling. You can create your custom dataset class by inheriting Dataset with some required member functions.

In [8]:
import torch
from torch.utils.data import DataLoader, TensorDataset

trainset = TensorDataset(torch.from_numpy(X_train_transformed.astype('float32')), torch.from_numpy(y_train.astype('float32')).view(-1,1))
trainloader = torch.utils.data.DataLoader(trainset, batch_size=4, shuffle=True, num_workers=2)

testset = TensorDataset(torch.from_numpy(X_test_transformed.astype('float32')), torch.from_numpy(y_test.astype('float32')).view(-1,1))
testloader = torch.utils.data.DataLoader(testset, batch_size=4, shuffle=False, num_workers=2)

Let's check some training samples

In [9]:
# get some random training samples
dataiter = iter(trainloader)
records, labels = dataiter.next()

print(records)
print(labels)


 0.0000  0.0000  0.0000  ...   0.0000  0.0000  0.0000
 0.0000  0.0000  0.0000  ...   0.0000  0.0000  0.0000
 0.0000  0.0000  0.0000  ...   0.0000  0.0000  0.0000
 0.0000  0.0000  0.0000  ...   0.0000  0.0000  0.0000
[torch.FloatTensor of size 4x9978]


 1
 1
 1
 0
[torch.FloatTensor of size 4x1]



#### 2. Define a Feed-forward Neural Network

In [10]:
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F


class FeedForwardNet(nn.Module):
    def __init__(self, n_input, n_hidden, n_output):
        super(FeedForwardNet, self).__init__()
        self.hidden1 = nn.Linear(n_input, n_hidden)
        self.hidden2 = nn.Linear(n_hidden, n_hidden)
        self.out = nn.Linear(n_hidden, n_output)

    def forward(self, x):
        x = F.relu(self.hidden1(x))
        x = F.relu(self.hidden2(x))
        x = self.out(x)
        return x

net = FeedForwardNet(n_input=9978, n_hidden=256, n_output=1)

#### 3. Define a Loss function and Optimizer
We will use Binary Cross Entropy loss and SGD with momentum as our optimizer.
PyTorch provide BCEWithLogitsLoss loss function which combines a Sigmoid layer and the BCEloss together and it is more numerically stable than using them separately. Keep in mind that you should not apply sigmoid activation after the output layer to use this combined loss.

In [11]:
import torch.optim as optim

criterion = nn.BCEWithLogitsLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

#### 4. Train the network

In [15]:
for epoch in range(2):  # loop over the dataset multiple times

    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        # get the inputs
        inputs, labels = data

        # wrap them in Variable
        inputs, labels = Variable(inputs), Variable(labels)

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.data[0]
        
        if i % 10 == 9:    # print every 10 mini-batches
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 10))
            running_loss = 0.0

print('Finished Training')

[1,    10] loss: 0.522
[1,    20] loss: 0.549
[1,    30] loss: 0.501
[1,    40] loss: 0.479
[1,    50] loss: 0.474
[1,    60] loss: 0.566
[2,    10] loss: 0.525
[2,    20] loss: 0.474
[2,    30] loss: 0.563
[2,    40] loss: 0.548
[2,    50] loss: 0.456
[2,    60] loss: 0.415
Finished Training
